# Build a deep neural network and use reinforcement learning to solve a cart and pole balancing problem using OpenAI

install OpenAI Gym


In [1]:
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [2]:
import gym
print(gym.__version__)
import keras
print(keras.__version__)

0.18.0
2.4.3


In [4]:
import random
import math
import numpy as np
from collections import deque

In [5]:
env=gym.make('CartPole-v0')

In [5]:
#setup the openAI gym envirement

env=gym.make('CartPole-v0')

for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        
        if done:
            print("episode finished after {} timesteps ".format(t+1))
            break

episode finished after 9 timesteps 
episode finished after 25 timesteps 
episode finished after 24 timesteps 
episode finished after 35 timesteps 
episode finished after 14 timesteps 
episode finished after 15 timesteps 
episode finished after 30 timesteps 
episode finished after 31 timesteps 
episode finished after 19 timesteps 
episode finished after 11 timesteps 
episode finished after 17 timesteps 
episode finished after 22 timesteps 
episode finished after 14 timesteps 
episode finished after 12 timesteps 
episode finished after 37 timesteps 
episode finished after 17 timesteps 
episode finished after 22 timesteps 
episode finished after 13 timesteps 
episode finished after 14 timesteps 
episode finished after 9 timesteps 


In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


In [7]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


##define Parameters

In [11]:
#training parameters
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gamma = 1.0 #Discount factor
epsilon = 1.0 #Exploration
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01 #the learning rate
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False 

#environement Parameters
memory = deque(maxlen = 100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None:
    env.max_episode_steps = max_env_steps

In [14]:
#building the neural Network

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))


In [15]:
#define necessary function
def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))
    
def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)*epslion_decay)))

def preprocess_state(state):
    return np.reshape(state, [1,4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predit(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch),verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
        

In [17]:
#Define Run function
def run():
    scores = deque(maxlen=100)
    
    for e in range(n_epsiodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1
            
        scores.append(i)
        mean_score = np.mean(scores)
        
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. solved after {} teails'.format(e, e-100))
            return e-100
        if e % 20 == 0 and not quiet:
            print('[episode {}]-Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
            
        replay(batch_size, epsilon)
    if not quiet:print('Did not solve after {} episodes'.format(e))
    return e
                
            

In [1]:
#Training the Network
import sys
import gym
import keras
import random
import math
import numpy as np
from collections import deque

#training parameters
n_episodes = 1000
n_win_ticks = 195
max_env_steps = None

gamma = 1.0 #Discount factor
epsilon = 1.0 #Exploration
epsilon_min = 0.01
epsilon_decay = 0.995
alpha = 0.01 #the learning rate
alpha_decay = 0.01

batch_size = 64
monitor = False
quiet = False 

#environement Parameters
memory = deque(maxlen = 100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None:
    env.max_episode_steps = max_env_steps
    
#building the neural Network

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

#model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))
    
#define necessary function
def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))
    
def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)*epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1,4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch),verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay
        
#Define Run function
def run():
    scores = deque(maxlen=100)
    
    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1
            
        scores.append(i)
        mean_score = np.mean(scores)
        
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. solved after {} teails'.format(e, e-100))
            return e-100
        if e % 20 == 0 and not quiet:
            print('[episode {}]-Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))
            
        replay(batch_size, epsilon)
    if not quiet:print('Did not solve after {} episodes'.format(e))
    return e
run()

[episode 0]-Mean survival time over last 100 episodes was 18.0 ticks.


KeyboardInterrupt: 